# Seeking out issues

In [1]:
import pandas as pd
import numpy as np

# pd.set_option('display.max_rows', 5000)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [49]:
def read(path):
    df = pd.read_csv(path, header=0, parse_dates=['pubTimeStamp'])
    return df

may = read('./data_scooter/may/may_trip.csv')
june = read('./data_scooter/june/june_trip.csv')
july = read('./data_scooter/july/july_trip.csv')

---

## Percentage of scooters reaching 3 or more rides a day

In [41]:
may1 = may.groupby([may.pubTimeStamp.dt.day, 'sumdID']).size()
june1 = may.groupby([may.pubTimeStamp.dt.day, 'sumdID']).size()
july1 = june.groupby([june.pubTimeStamp.dt.day, 'sumdID']).size()

In [42]:
may_gt_3 = (may1 >= 3).sum(); may_lt_3 = (may1 < 3).sum()

june_gt_3 = (june1 >= 3).sum(); june_lt_3 = (june1 < 3).sum()

july_gt_3 = (july1 >= 3).sum(); july_ls_3 = (july1 < 3).sum()

may_met_3 = (may_gt_3 / (may_gt_3 + may_lt_3) * 100)
june_met_3 = (june_gt_3 / (june_gt_3 + june_lt_3) * 100)
july_met_3 = (july_gt_3 / (july_gt_3 + july_ls_3) * 100)

f"May: {round(may_met_3, 2)}%"
f"June: {round(june_met_3, 2)}%"
f"July: {round(july_met_3, 2)}%"
f"Average: {round(np.mean([may_met_3, june_met_3, july_met_3]))}%"

'May: 43.6%'

'June: 43.6%'

'July: 40.81%'

'Average: 43.0%'

## How many unique sumdID's? 

`Slide says 5860`  
`Below is 9005`


In [57]:
all_months = pd.concat([may, june, july])
all_months.shape[0]
may.shape[0]+june.shape[0]+july.shape[0] # double checking everything concatted

565522

565522

In [26]:
print(f"May unique sumdID's: {may.reset_index().sumdID.nunique()}")
print(f"June unique sumdID's: {june.reset_index().sumdID.nunique()}")
print(f"July unique sumdID's: {july.reset_index().sumdID.nunique()}\n")

print(f"Unique sumdID's across all months: {all_months.reset_index().sumdID.nunique()}")

May unique sumdID's: 6966
June unique sumdID's: 6742
July unique sumdID's: 5585

Unique sumdID's across all months: 9005


## Each month dataset has data from succeeding month

Cases not native to the datasets:
- `may`: 466  
- `June`: 112  
- `July`: 78 

In [70]:
# filtering out 
may2 = may[may.pubTimeStamp.dt.month == 5]
june2 = june[june.pubTimeStamp.dt.month == 6]
july2 = july[july.pubTimeStamp.dt.month == 7]
may2.shape[0]; june2.shape[0]; july2.shape[0]

224503

205515

134848

## Aren't trip record numbers supposed to be unique?

The assumption is trips are supposed to be unique?  
Look what I found below.

In [133]:
# may2grouped = may2.groupby(may2.pubTimeStamp.dt.day)['tripRecordNum'].value_counts()
may2grouped = may2.groupby(['sumdID','tripRecordNum']).size()

ununique_tripnums = (may2grouped[may2grouped > 1]
                     .index
                     .get_level_values(1)
                     .to_list())

ununique_trip_df = may2[may2['tripRecordNum'].isin(ununique_tripnums)]
ununique_trip_df.shape[0]
may2.shape[0]

42425

224503

In [132]:
ununique_trip_df.companyName.value_counts()

Lime             25333
Lyft             12215
Bird              3576
SPIN               700
JUMP               224
Bolt Mobility      216
Gotcha             161
Name: companyName, dtype: int64

In [143]:
may3grouped = may2.groupby([may2.pubTimeStamp.dt.day, 'tripRecordNum']).size()
june3grouped = june2.groupby([june2.pubTimeStamp.dt.day, 'tripRecordNum']).size()
july3grouped = july2.groupby([july2.pubTimeStamp.dt.day, 'tripRecordNum']).size()

In [146]:
(may3grouped > 1).sum()
may3grouped.shape
(june3grouped > 1).sum()
june3grouped.shape
(july3grouped > 1).sum()
july3grouped.shape

7750

(216753,)

1576

(203939,)

375

(134473,)

In [142]:
may3grouped.value_counts()

1    209003
2      7750
dtype: int64

In [148]:
may2.columns

Index(['pubTimeStamp', 'companyName', 'tripRecordNum', 'sumdID',
       'tripDuration', 'tripDistance', 'startDate', 'startTime', 'endDate',
       'endTime', 'startLatitude', 'startLongitude', 'endLatitude',
       'endLongitude', 'tripRoute', 'create_dt'],
      dtype='object')

In [153]:
sum(may2.tripDistance < 3)

19945